### Librerias

In [1]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
from multiprocessing import Process
import threading
import time
import numpy as np


2023-02-06 08:56:45.974867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 08:56:46.693222: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lenin/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-02-06 08:56:46.693256: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 08:56:50.178794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerro

### funciones para preprocesar la img

In [2]:
def redimensionar(img, nuevo_ancho, nuevo_alto):
    puntos_bajar = (nuevo_ancho, nuevo_alto)
    new_img = cv2.resize(img, puntos_bajar, interpolation= cv2.INTER_LINEAR)
    return new_img

In [3]:
def guardar_frame(frame):
    Datos='/home/lenin/Documents/GitHub/Modelo-para-clasificacion-de-colores-ANN/imgs/frame.jpg'
    cv2.imwrite(Datos, frame)

In [4]:
def guardar_imgs(imgs):
    try:
        for i in range(len(imgs)):
            Datos='/home/lenin/Documents/GitHub/Modelo-para-clasificacion-de-colores-ANN/imgs/img{}.jpg'.format(i)
            cv2.imwrite(Datos, imgs[i])
    except Exception as e:
        print(f'error en gudardar img: {e}')

In [5]:
def dividir_img(img,nuevo_ancho, nuevo_alto, n_filas, n_column):
#     nuevo_ancho = 200
#     nuevo_alto = 200

    new_img = redimensionar(img, nuevo_ancho, nuevo_alto)
    # guardar_frame(new_img)
    img_base = new_img
    imgs=[]
    
    height, width, channels = new_img.shape
    CROP_W_SIZE  = n_filas
    CROP_H_SIZE = n_column

    for ih in range(CROP_H_SIZE ):
        for iw in range(CROP_W_SIZE ):
            x = width//CROP_W_SIZE * iw 
            y = height//CROP_H_SIZE * ih
            h = (height // CROP_H_SIZE)
            w = (width // CROP_W_SIZE )
            yy = int(y+h)
            xx = int(x+w)
            temporal = img_base[y:yy, x:xx]
            imgs.append(temporal)
            img_base = new_img
    guardar_imgs(imgs)
    return imgs

In [6]:
def predecir_imgs(modelo, imgs, vector):
    try:
        print('\n\n')
        clases = [] 
        labels = []
        for i in range(len(imgs)):
            img = imgs[i]
            img = img_to_array(img)
            img = img/255       
            img = img[np.newaxis, ...]
            respuesta = modelo.predict(img, verbose=0)
            posicion = np.argmax(respuesta[0])
            print(f' label: {posicion}  clase: {vector[posicion]}')
    except Exception as e:
        print(f'error en predecir imgs {e}')
    return clases

In [7]:
def adjust_brightness(img, alpha, beta):
    return cv2.addWeighted(img, alpha, np.zeros(img.shape, img.dtype), 0, beta)

In [8]:
def metodo_resizing(modelo, frame, nuevo_size, celda, vector):
    try:
        imgs = dividir_img(frame, nuevo_size, nuevo_size, celda, celda)
        # guardar_imgs(imgs)
        clases = predecir_imgs(modelo, imgs, vector)
    except Exception as e:
        print("Error en metodo_resizing:", e)

In [9]:
def metodo_pixel_aleatorio(modelo, frame, vector, num_pixeles):
    try:
        # frame = redimensionar(frame, 50, 50)
        guardar_frame(frame)
        height, width = frame.shape[:2]
        coordenadas = np.random.randint(0, height, (num_pixeles, 2))
        imgs=[]
        for (m,n) in coordenadas:
            pixel = frame[m,n]
            pixel = pixel.reshape(3, 3, 3)
            # pixel = redimensionar(pixel, 5, 5)
            # print(pixel.shape)
            imgs.append(pixel)
        guardar_imgs(imgs)
        # print(f"\r{coordenadas}", end=" ")   
        clases = predecir_imgs(modelo, imgs, vector)
        print(f"\r{clases}", end=" ")
        del imgs, frame, height, width, num_pixeles, coordenadas, clases
    except Exception as e:
        print(f'error en metodo pixel aleatorio: {e}')

### Preparo cv2

In [15]:
# 0 camara predeterminada
cap = cv2.VideoCapture(0)
vector = ['morado', 'azul', 'rojo', 'amarillo', 'gris', 'rosado', 'negro', 'naranja', 'celeste', 'verde', 'blanco', 'cafe']
modelo = tf.keras.models.load_model("mod_color_v1_3x3.h5")
modelo.load_weights("mod_color_v1_weights_3x3.h5")
nuevo_size = 9
celda = nuevo_size // 3
print(f'celda {celda}')

c=0
while True:
    # Capturar un marco
    ret, frame = cap.read()
    #volteo el frame
    frame = cv2.flip(frame, 1)
    # ajusto brillo
    frame = adjust_brightness(frame, 1.3, 3)
    # muestro la ventana
    cv2.imshow('frame', frame)
    # Salir si se presiona 'esc'
    key = cv2.waitKey(1)
    if key == 27:
        break
    # continue
    if c >= 60:
        c=0
        # predecir(frame)
        # print(frame.shape)
        metodo_resizing(modelo, frame, nuevo_size, celda, vector)
        # metodo_pixel_aleatorio(modelo, frame, vector, 4)
        # break
    c+=1
cap.release()
cv2.destroyAllWindows()


celda 3



 label: 4  clase: gris
 label: 11  clase: cafe
 label: 11  clase: cafe
 label: 4  clase: gris
 label: 8  clase: celeste
 label: 4  clase: gris
 label: 11  clase: cafe
 label: 11  clase: cafe
 label: 11  clase: cafe



 label: 4  clase: gris
 label: 11  clase: cafe
 label: 11  clase: cafe
 label: 1  clase: azul
 label: 4  clase: gris
 label: 4  clase: gris
 label: 11  clase: cafe
 label: 4  clase: gris
 label: 11  clase: cafe



 label: 4  clase: gris
 label: 11  clase: cafe
 label: 4  clase: gris
 label: 11  clase: cafe
 label: 4  clase: gris
 label: 1  clase: azul
 label: 11  clase: cafe
 label: 11  clase: cafe
 label: 4  clase: gris



 label: 5  clase: rosado
 label: 4  clase: gris
 label: 4  clase: gris
 label: 4  clase: gris
 label: 4  clase: gris
 label: 4  clase: gris
 label: 8  clase: celeste
 label: 1  clase: azul
 label: 4  clase: gris



 label: 5  clase: rosado
 label: 5  clase: rosado
 label: 4  clase: gris
 label: 5  clase: rosado
 label: 11  clase: cafe
 label